In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

2024-06-20 16:11:52.303915: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 16:11:52.304108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 16:11:52.460469: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
csv_path = "/kaggle/input/devanagari-character-set/data.csv"

In [3]:
data = pd.read_csv(csv_path)

In [4]:
data.shape

(92000, 1025)

In [5]:
data.head()

,pixel_0000,pixel_0001,pixel_0002,pixel_0003,pixel_0004,pixel_0005,pixel_0006,pixel_0007,pixel_0008,pixel_0009,...,pixel_1015,pixel_1016,pixel_1017,pixel_1018,pixel_1019,pixel_1020,pixel_1021,pixel_1022,pixel_1023,character
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,character_01_ka
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,character_01_ka
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,character_01_ka
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,character_01_ka
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,character_01_ka


In [6]:
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Target

In [7]:
X.shape

(92000, 1024)

In [8]:
y.shape

(92000,)

In [9]:
num_classes = len(np.unique(y))
y = pd.factorize(y)[0]  # Convert character labels to integer
y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

In [10]:
 X = np.random.random((92000, 1024))

In [11]:
def functional_mlp():
   
    inp= Input(shape=(X.shape[1],))
    a =  Dense(units=512, activation="relu")(inp)
    b =  Dense(units=256, activation="relu")(a)
    c=  Dense(units=128, activation="relu")(b)
    out=  Dense(units=46, activation="softmax")(c)

    return Model(inputs=inp, outputs=out)

In [12]:
functional_nn = functional_mlp()
functional_nn.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 46)             │         5,934 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 694,958 (2.65 MB)

 Trainable params: 694,958 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
def categorical_crossentropy_loss(y_true, y_pred):
    y_true = tf.cast(y_true, y_pred.dtype)
    # Categorical crossentropy loss
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true=y_true, y_pred=y_pred))
    return loss

In [14]:
epochs = 10
optimizer =tf.keras.optimizers.Adam(learning_rate=0.0001)
batch_size = 1024

In [15]:
for epoch in range(epochs):
    # Shuffle data
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X_shuffled = X[indices]
    y_shuffled = y[indices]

In [16]:
for epoch in range(epochs):
    for i in range(0, len(X), batch_size):
        batch_X = X_shuffled[i:i+batch_size]
        batch_y = y_shuffled[i:i+batch_size]

        with tf.GradientTape() as tape:
            Y_pred = functional_nn(batch_X, training=True)
            loss_func = categorical_crossentropy_loss(batch_y, Y_pred)
            loss_func = tf.reduce_mean(loss_func)

        gradients = tape.gradient(loss_func, functional_nn.trainable_weights)
        grads_and_vars = zip(gradients, functional_nn.trainable_weights)  
        optimizer.apply_gradients(grads_and_vars)
    
    print("Epoch # {}, Loss Function Value is = {}".format(epoch+1, loss_func))

        

Epoch # 1, Loss Function Value is = 3.8309597969055176
Epoch # 2, Loss Function Value is = 3.8270366191864014
Epoch # 3, Loss Function Value is = 3.8250608444213867
Epoch # 4, Loss Function Value is = 3.8228700160980225
Epoch # 5, Loss Function Value is = 3.8195834159851074
Epoch # 6, Loss Function Value is = 3.8176777362823486
Epoch # 7, Loss Function Value is = 3.8174335956573486
Epoch # 8, Loss Function Value is = 3.8098113536834717
Epoch # 9, Loss Function Value is = 3.808013916015625
Epoch # 10, Loss Function Value is = 3.8007376194000244
